In [1]:
%matplotlib inline

In [2]:
#export
from exp.nb_03 import *

# Data Setup

In [3]:
xTraining, yTraining, xValidation, yValidation = getMnistData()

In [4]:
xTrainingNormalized = normalizeVector(xTraining, xTraining.mean(), xTraining.std())
xValidationNormalized = normalizeVector(xValidation, xValidation.mean(), xValidation.std())

In [5]:
((xTrainingNormalized.std(), xTrainingNormalized.mean()), (xValidationNormalized.std(), xValidationNormalized.mean()))

((tensor(1.), tensor(-7.6999e-06)), (tensor(1.0000), tensor(-7.0751e-08)))

In [6]:
trainingDataSet = Dataset(xTrainingNormalized, yTraining)
validationDataSet = Dataset(xValidationNormalized, yValidation)

In [7]:
hiddenLayerOutput, batchSize = 50, 64

In [8]:
numberOfClasses = yValidation.max().item() + 1

In [9]:
trainingRows, trainingColumns = xTrainingNormalized.shape

In [10]:
learningRate = 0.5

In [11]:
selfLearningModel = SelfLearningLibraryModel(trainingColumns,hiddenLayerOutput,numberOfClasses,learningRate)

In [12]:
#export
class DataBunch():
    def __init__(self, trainingDataLoader, validationDataLoader):
        self.trainingDataLoader, self.validationDataLoader = trainingDataLoader, validationDataLoader
        
    @property
    def trainingDataSet(self): return self.trainingDataLoader
    
    @property
    def validationDataSet(self): return self.validationDataLoader

In [13]:
trainingDataLoader, validationDataLoader = createDataLoaders(trainingDataSet,
                                                            validationDataSet, 5000)

In [14]:
imageDataBunch = DataBunch(trainingDataLoader, validationDataLoader)

In [114]:
#export
def trainModelWithValidation(learnableModel, dataBunch, epochs):
    for epoch in range(epochs):
        for _xTrain, _yTrain in dataBunch.trainingDataLoader:
            _preds = learnableModel(_xTrain)
            loss = Functional.cross_entropy(_preds, _yTrain)
            loss.backward()
            learnableModel.learn()
        accumulatedLoss, accumulatedAccuracy = 0.,0.
        for _xValidation, _yValidation in dataBunch.validationDataLoader:
            with torch.no_grad(): # do not produce gradients when running loss function
                _preds = learnableModel(_xValidation)
                accumulatedLoss += Functional.cross_entropy(_preds, _yValidation)
                accumulatedAccuracy += accuracy(_preds, _yValidation)
        numberOfItems = len(validationDataLoader)
        print("Epoch {}, Accuracy {}, Loss {}".format(epoch, accumulatedAccuracy/numberOfItems, accumulatedLoss/numberOfItems))

In [16]:
accuracy(selfLearningModel(xValidationNormalized), yValidation)

tensor(0.0786)

In [17]:
trainModelWithValidation(selfLearningModel, imageDataBunch, 9)

Epoch 0, Accuracy 0.33469998836517334, Loss 1.814199447631836
Epoch 1, Accuracy 0.6765000224113464, Loss 0.9899466037750244
Epoch 2, Accuracy 0.8217999935150146, Loss 0.5179767608642578
Epoch 3, Accuracy 0.8913999795913696, Loss 0.3701602816581726
Epoch 4, Accuracy 0.8654000163078308, Loss 0.39331528544425964
Epoch 5, Accuracy 0.9200000166893005, Loss 0.27125871181488037
Epoch 6, Accuracy 0.9254000186920166, Loss 0.25498273968696594
Epoch 7, Accuracy 0.9241999983787537, Loss 0.24994486570358276
Epoch 8, Accuracy 0.9312000274658203, Loss 0.22827160358428955


In [18]:
accuracy(selfLearningModel(xValidationNormalized), yValidation)

tensor(0.9312)

## Other Stuff

Things that facilitate the trainging of models

In [115]:
class Teacher():
    
    def __init__(self, lossFunction, accuracyFunction):
        self.lossFunction = lossFunction
        self.accuracyFunction = accuracyFunction
        
    def teachModel(self, selfLearningModel, dataBunch, numberOfEpochs):
        for epoch in range(numberOfEpochs):
            trainingLoss, trainingAccuracy = self._trainModel(selfLearningModel, dataBunch.trainingDataLoader)
            validationLoss, validationAccuracy = self._validateModel(selfLearningModel, dataBunch.validationDataLoader)
            print("Epoch #{} Training: Loss {} Accuracy {}".format(epoch, trainingLoss, trainingAccuracy))
            print("Epoch #{} Validation: Loss {} Accuracy {}".format(epoch, validationLoss, validationAccuracy))
            print("")
    
    def _proccessDataSet(self, selfLearningModel, dataLoader, isTeaching=False):
        for _xDataSet, _yDataSet in dataLoader:
            _predictions = selfLearningModel(_xDataSet)
            loss = self.lossFunction(_predictions, _yDataSet)
            
            if(isTeaching):
                loss.backward()
                selfLearningModel.learn()
            
            return loss, self.accuracyFunction(_predictions, _yDataSet)
            
    def _trainModel(self, selfLearningModel, trainingDataSet):
        returnItems = self._proccessDataSet(selfLearningModel, trainingDataSet, True)
        return returnItems
        
    def _validateModel(self, selfLearningModel, validationDataSet):
        with torch.no_grad():
            returnItems = self._proccessDataSet(selfLearningModel, validationDataSet)
        return returnItems

In [110]:
teacher = Teacher(Functional.cross_entropy, accuracy)

In [111]:
_newSelfLearningModel = SelfLearningLibraryModel(trainingColumns, hiddenLayerOutput,numberOfClasses,learningRate)

In [112]:
teacher.teachModel(_newSelfLearningModel, imageDataBunch, 8)

Epoch #0 Training: Loss 2.334801435470581 Accuracy 0.05820000171661377
Epoch #0 Validation: Loss 1.8325834274291992 Accuracy 0.45329999923706055

Epoch #1 Training: Loss 1.8481521606445312 Accuracy 0.42739999294281006
Epoch #1 Validation: Loss 1.6057158708572388 Accuracy 0.4884999990463257

Epoch #2 Training: Loss 1.619502305984497 Accuracy 0.47859999537467957
Epoch #2 Validation: Loss 2.037757396697998 Accuracy 0.4733999967575073

Epoch #3 Training: Loss 2.0454702377319336 Accuracy 0.4803999960422516
Epoch #3 Validation: Loss 1.472883939743042 Accuracy 0.5480999946594238

Epoch #4 Training: Loss 1.5267990827560425 Accuracy 0.5311999917030334
Epoch #4 Validation: Loss 1.3862677812576294 Accuracy 0.5084999799728394

Epoch #5 Training: Loss 1.4516279697418213 Accuracy 0.4860000014305115
Epoch #5 Validation: Loss 1.3181079626083374 Accuracy 0.578000009059906

Epoch #6 Training: Loss 1.3737976551055908 Accuracy 0.550000011920929
Epoch #6 Validation: Loss 1.634998083114624 Accuracy 0.493299

In [113]:
accuracy(_newSelfLearningModel(xValidationNormalized), yValidation)

tensor(0.4865)

In [83]:
trainModelWithValidation(_newSelfLearningModel, imageDataBunch, 8)

Epoch 0, Accuracy 0.6759999990463257, Loss 0.8778393268585205
Epoch 1, Accuracy 0.8406999707221985, Loss 0.5532947182655334
Epoch 2, Accuracy 0.8866999745368958, Loss 0.3865782916545868
Epoch 3, Accuracy 0.9093000292778015, Loss 0.3086942732334137
Epoch 4, Accuracy 0.9115999937057495, Loss 0.2992865741252899
Epoch 5, Accuracy 0.9258999824523926, Loss 0.2472270280122757
Epoch 6, Accuracy 0.9190000295639038, Loss 0.2661859393119812
Epoch 7, Accuracy 0.9337999820709229, Loss 0.22734184563159943


In [84]:
accuracy(_newSelfLearningModel(xValidationNormalized), yValidation)

tensor(0.9338)

In [116]:
!python notebook2script.py 04_HOLLA_BACK_GURL.ipynb

Converted 04_HOLLA_BACK_GURL.ipynb to exp/nb_04.py
